In [ ]:
# Most up-to-date version of H5Coro needed to take NSIDC credentials
%pip install -U h5coro

In [ ]:
# For searching and accessing NASA data
import earthaccess

# For reading data, analysis and plotting
import xarray as xr
import hvplot.xarray
from h5coro import h5coro, s3driver
import numpy as np
import pandas as pd
import geopandas as gpd

import pprint  # For nice printing of python objects

In [ ]:
earthaccess.login(strategy='interactive', persist=True)

In [ ]:
auth = earthaccess.login()

In [ ]:
gj = gpd.read_file('/home/jovyan/icesat2-snowex/jsons-shps/toolik_lidar_boxes.geojson')
bbox = tuple(list(gj.total_bounds))

In [ ]:
short_name = "ATL12"

results = earthaccess.search_data(
    short_name = short_name,
    cloud_hosted = True,
    bounding_box = bbox,
    temporal = ('2022-01-01','2022-01-31')
)

In [ ]:
%time
files = earthaccess.open(results)

In [ ]:
s3_creds = auth.get_s3_credentials(daac="NSIDC")

In [ ]:
granule = files[0]
granule.full_name

In [ ]:
h5obj = h5coro.H5Coro(granule.full_name[5:],
                      s3driver.S3Driver, credentials=s3_creds)

In [ ]:
if short_name == "ATL03":
    # ATL03
    variables = ['gt2l/heights/lat_ph',
                 'gt2l/heights/lon_ph',
                 'gt2l/heights/signal_conf_ph',
                 'gt2l/heights/h_ph'
                 ]
    print("Caution: File is very large. Consider spatial subsetting or photon filtering "\
          "before converting to a geodataframe.")
elif short_name == "ATL06":
    # ATL06
    variables = ['gt2l/land_ice_segments/latitude',
                 'gt2l/land_ice_segments/longitude',
                 'gt2l/land_ice_segments/h_li'
                 ]
elif short_name == "ATL07":
    # ATL07
    variables = ['gt2l/sea_ice_segments/latitude',
                 'gt2l/sea_ice_segments/longitude',
                 'gt2l/sea_ice_segments/heights/height_segment_height'
                 ]
elif short_name == "ATL08":
    # ATL08
    variables = ['gt2l/land_segments/latitude',
                 'gt2l/land_segments/longitude',
                 'gt2l/land_segments/terrain/h_te_best_fit'
                 ]
elif short_name == "ATL12":
    # ATL12
    variables = ['gt2l/ssh_segments/latitude',
                 'gt2l/ssh_segments/longitude',
                 'gt2l/ssh_segments/heights/h'
                 ]
else:
    print("Unrecognized product short name.")

In [ ]:
promise = h5obj.readDatasets(variables, block=True, enableAttributes=False)

In [ ]:
var_paths = list(promise.keys())

In [ ]:
df = pd.DataFrame(data={'lat': promise[var_paths[0]],
                        'lon': promise[var_paths[1]],
                        'height': promise[var_paths[2]]
                       }
                 )
df.head()

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat), crs='epsg:4326')
gdf

In [ ]:
gdf.explore(column="height", tiles="Esri.WorldImagery")